네이버(https://www.naver.com)에서 뉴스 기사를 검색하기 위한 크롤러 입니다.  
검색명, 검색기간, 언론사, 정렬방법을 설정할 수 있습니다.    

In [1]:
'''--- 공통(beautiful soup, selenium) ---'''
import os
import sys
import time
from datetime import datetime    # 날짜 모듈
from tqdm import tqdm_notebook   # 진행 상황을 bar로 시각화

import pandas as pd
import numpy as np
import re

'''--- beautiful soup ---'''
from bs4 import BeautifulSoup

'''--- Selenium webdriver ---'''
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

'''--- 저장 ---'''
import csv  

In [2]:
# 현재 jupyter 위치를 PROJECT_DIR 추가
PROJECT_DIR = os.path.abspath(os.path.join(os.path.realpath('__file__'), '..'))
sys.path.insert(0, PROJECT_DIR)
DATA_DIR = PROJECT_DIR + '/data' # 데이터 위치 
# print(DATA_DIR)

## function

In [3]:
def set_period(driver, start_date, end_date):
    '''
    selenium webdriver로 crawling 진행 시 "검색기간"을 설정해주는 함수입니다.
    
    Parameters: 
    ---------- 
    driver : selenium.webdriver
    start_date: str
        검색 시작 일자
    end_date : str
        검색 마지막 일자
    '''
       
    # - btn: '기간'
    driver.find_element(By.XPATH, '//*[@id="snb"]/div/ul/li[2]/a').click() 
    time.sleep(0.5)
     
    # - input : 검색 기간
    input_sd = driver.find_element_by_id('news_input_period_begin')  # start_date
    input_ed = driver.find_element_by_id('news_input_period_end')    # end_date
    input_sd.send_keys(str(start_date))
    input_ed.send_keys(str(end_date))
    print('--기간입력--')
    time.sleep(0.5)
    
    # - btn : '적용'
    driver.find_element_by_xpath('//*[@id="snb"]/div/ul/li[2]/div/div[2]/span/button/span').click()
    print('적용')

In [24]:
def set_media(driver, media_list):
    '''
    selenium webdriver로 crawling 진행 시 "언론사"를 설정해주는 함수입니다.
    
    Parameters: 
    ---------- 
    driver : selenium.webdriver
    media_list : dict {'media name':'Xpath'}
        언론사 리스트
    
    '''
    
    # - btn : '언론사'
    driver.find_element(By.XPATH, '//*[@id="snb"]/div/ul/li[5]/a').click()
    print('--언론사 설정--')
    # - check box : 언론사 
    for s in media_list:
        element = driver.find_element(By.XPATH, media_list[s])
        driver.execute_script("arguments[0].click();", element)
        print('media:', s)
        time.sleep(0.5)
    # - btn : 확인
    driver.find_element(By.XPATH, '//*[@id="snb"]/div/ul/li[5]/div/span/span[1]/button').click()
    print('확인')            

In [5]:
def set_seachKey_sort(search_key, sort_num):
    '''
    "검색어"와 검색결과 "정렬방법"을 설정해주는 함수입니다. 
    
    Parameters: 
    ---------- 
    search_key : str
        검색어
    sort_num : int
        0.관련된 순, 1.최신순, 2.오래된 순
    
    Returns: 
    ------- 
    url : str
        네이버에서 검색어와 정렬방법이 적용된 url
    '''
        
    url='https://search.naver.com/search.naver?&where=news&query='+search_key\
    +'&sm=tab_srt&sort='+str(sort_num)+'&photo=0&field=1&reporter_article=&pd=3&ds=2020.11.01&de=2020.12.31&docid=&nso=so%3Ada%2Cp%3Afrom20201101to20201231%2Ca%3At&mynews=1&refresh_start=1&related=0'
    
    return url


In [6]:
def set_url(cur_url):
    '''
    selenium webdriver로 진행할 때, 계속 버튼 없이 다음 페이지로 접근하기 위하여
    contents number를 제외한 앞, 뒤 url를 담은 list를 반환해주는 함수입니다. 
    
    Parameters: 
    ---------- 
    cur_url : driver.current_urlcurrent
      
    Returns: 
    ------- 
    default_url : list[str,str]
        contents number 제외한 앞, 뒤 url
        
    '''
    
    url_t = cur_url.split('&mynews')
    url_a = ''.join([url_t[0],'&mynews=1&start='])
    default_url = [url_a, '&refresh_start=1']
    
    return default_url
    

In [7]:
def load_url(default_url, cont_num):   # default_url : type: list
    '''
    Parameters: 
    ---------- 
    default_url : list[str,str]
        set_url 함수 결과로 contents number를 제외한 앞, 뒤 url 리스트
    cont_num : int
        contents number
    
    Returns: 
    ------- 
    url : str
        content number가 포함된 완성된 url
    
    '''

    url =default_url[0] + str(cont_num) + default_url[1]
    
    
    return url

In [8]:
def clear_content(text, media_list):
    '''
    뉴스 내용을 정제하는 함수입니다.
    
    Parameters: 
    ---------- 
    text : str
        크롤링한 뉴스 내용    
    media_list : dict {'media name':'Xpath'}
        언론사 리스트
        
    Returns: 
    ------- 
    cleared_content :  str
        정제된 뉴스 내용
        
    '''
    
    remove_special = re.sub('[∙©\{\}\[\]\/?,;:|\)*~`!^\-_+<>@\#$%&▲▶◆◀■\\\=\(\'\"]', '', text)
    remove_flash_error = re.sub('본문 내용|TV플레이어| 동영상 뉴스|flash 오류를 우회하기 위한 함수 추가fuction flashremoveCallback|tt||앵커 멘트|xa0', '', remove_special)
    remove_strip = remove_flash_error.strip().replace('   ', '') # 공백 에러 삭제
    remove_dphase = re.sub('if deployPhase(.*)displayRMCPlayer', '', remove_strip)
    
    media_list = '|'.join(media_list)  
    remove_media = re.sub(media_list,'', remove_dphase)  # 언론사명 삭제
    reversed_content = ''.join(reversed(remove_special))  # 기사 내용을 reverse 한다.
    content = ''
    for i in range(0, len(reversed_content)):
    # reverse 된 기사 내용중, ".다"로 끝나는 경우 기사 내용이 끝난 것이기 때문에 기사 내용이 끝난 후의 광고, 기자 등의 정보는 다 지움
        if reversed_content[i:i + 2] == '.다':
            content = ''.join(reversed(reversed_content[i:]))
            break   
        
    cleared_t = re.sub('\t','',content)
    cleared_n = re.split('\n',cleared_t)
    cleared_content = ' '.join([ s for s in cleared_n])
    cleared_content = cleared_content.replace('  ',' ')
    cleared_content = re.sub(r'[^A-Za-z0-9가-힣,. ]','',cleared_content)

    return cleared_content

In [9]:
def get_media(bs):
    '''
    해당 기사의 언론사명을 반환해주는 함수입니다.
    
    Parameters: 
    ---------- 
    bs : BeautifulSoup
    
    Returns: 
    ------- 
    media : str
        언론사명
        
    '''
    
    media = bs.find('div', {'class':'press_logo'}).find('img')['alt']
    return media

In [10]:
def get_date(bs):
    '''
    기사 입력일을 반환해주는 함수입니다.
    
    Parameters: 
    ---------- 
    bs : BeautifulSoup
    
    Returns: 
    ------- 
    date : str
        날짜(0000-00-00)
        
    '''
    
    date = bs.find('span',{'class':['t11','author']})
    date = '-'.join(re.findall('\d+',date.text)[:3]) 
    return date

In [11]:
def get_title(bs):
    '''
    기사 제목을 반환해주는 함수입니다.
    
    Parameters: 
    ---------- 
    bs : BeautifulSoup
    
    Returns: 
    ------- 
    title : str
        기사 제목
    '''
    title=''
    if bs.find('h3',{'id':'articleTitle'}):
        title = bs.find('h3',{'id':'articleTitle'})
    elif bs.find('h2',{'class':'end_tit'}):
        title = bs.find('h2',{'class':'end_tit'})
    else:
        pass
    title = re.sub(r'[^A-Za-z0-9가-힣 ]','',title.text)  #cleaning
    return title

In [12]:
def get_content(bs, media_list):
    '''
    뉴스 본문을 반환해주는 함수입니다. 
    
    Parameters: 
    ---------- 
    bs : BeautifulSoup
    media_list : dict {'media name':'Xpath'}
        언론사 리스트
    
    Returns: 
    ------- 
    article : str
        뉴스 본문 텍스트
    
    '''
    
    article_obj =bs.find_all('div')[0] 

    # 본문 부분 추출
    if article_obj.find('div',{'id':'articleBodyContents'}):
        article_obj = article_obj.find('div',{'id':'articleBodyContents'})
    elif article_obj.find('div',{'class': 'article_body'}):
        article_obj = article_obj.find('div',{'id':'articeBody'})

    # tag 정제
    for e in range(len(article_obj.find_all('em'))):
        article_obj.em.extract()
        article_obj.img.extract()
    
    for i in range(len(article_obj.find_all('a'))):
        article_obj.a.extract()

    if article_obj.find('script'):
        article_obj.script.extract()
    if article_obj.find('strong'):
        article_obj.strong.extract()

    # 텍스트 정제
    article_obj = BeautifulSoup(re.split('\w\w\w 기자',article_obj.prettify())[0], 'lxml')
    article = clear_content(article_obj.text, media_list)

    
    return article

In [13]:
def news_parsing(url, media_list):
    '''
    해당 url에서 언론사, 기사입력일, 기사제목, 기사내용, url을 하나의 리스트로 반환해주는 함수입니다. 
    
    Parameters: 
    ---------- 
    url : str
        신문기사 페이지 url
    media_list : dict {'media name':'Xpath'}
        언론사 리스트

    Returns: 
    ------- 
    news : list [media, date, title, content, url]
        언론사, 기사입력일, 기사제목, 기사내용, url 리스트
    
    '''
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'}
    res = requests.post(url, headers=headers)
    bs_obj_n = BeautifulSoup(res.text, 'lxml')

    media_list = list(media_list.keys())
    news = []
    news.append(get_media(bs_obj_n))                 # media (언론사)
    news.append(get_date(bs_obj_n))                  # date (기사 입력일)
    news.append(get_title(bs_obj_n))                 # title
    news.append(get_content(bs_obj_n, media_list))   # content
    news.append(url)                                 # url
    
    return news

## main

In [25]:
# webdriver open
path = '../lib/chromedriver.exe'
driver = webdriver.Chrome(path)

In [26]:
# 검색 조건 설정
search_key = 'BTS | 방탄소년단'
sort_num = '2'   # 0.관련된 순, 1.최신순, 2.오래된 순
start_date = '2020.01.01'
end_date = '2020.12.30'
media_list = {'경향신문':'//*[@id="ca_1032"]', '중앙일보':'//*[@id="ca_1025"]','한겨례':'//*[@id="ca_1028"]','조선일보':'//*[@id="ca_1023"]'}

In [27]:
driver.get(set_seachKey_sort(search_key, sort_num))  # 검색어, 정렬순 적용된 1 페이지 접속
time.sleep(0.5)

In [28]:
set_period(driver, start_date, end_date)    

--기간입력--
적용


In [29]:
set_media(driver, media_list)

--언론사 설정--
media: 경향신문
media: 중앙일보
media: 한겨례
media: 조선일보
확인


In [30]:
default_url = set_url(driver.current_url)
default_url

['https://search.naver.com/search.naver?where=news&query=BTS%20%7C%20%EB%B0%A9%ED%83%84%EC%86%8C%EB%85%84%EB%8B%A8&sm=tab_opt&sort=2&photo=0&field=1&reporter_article=&pd=3&ds=2020.01.01&de=2020.12.30&docid=&nso=so%3Ada%2Cp%3Afrom20200101to20201230%2Ca%3At&mynews=1&start=',
 '&refresh_start=1']

In [31]:
# 웹페이지 순서대로 탐색
num = 1
link_list =[]
naver = 'news.naver.com/main/'

while(1):
    print('num:',num)
    url = load_url(default_url,num)
    driver.get(url)
    # 페이지(html) 가져오기
    page_html = driver.page_source  # type: str
    bs_obj = BeautifulSoup(page_html, 'lxml')
    # 뉴스 10개 link list
    lis = [ a['href'] for a in bs_obj.find_all('a', href=True) if naver in a['href'] ]
    if len(lis)==0:
        print('len(lis):',len(lis))
        break
    link_list.extend(lis)
    time.sleep(0.5)
    num+=10
    
print('len(link_list):',len(link_list))
# link_list

num: 1
num: 11
num: 21
num: 31
num: 41
num: 51
num: 61
num: 71
num: 81
num: 91
num: 101
num: 111
num: 121
num: 131
num: 141
num: 151
num: 161
num: 171
num: 181
num: 191
num: 201
num: 211
num: 221
num: 231
num: 241
num: 251
num: 261
num: 271
num: 281
num: 291
num: 301
num: 311
num: 321
num: 331
num: 341
num: 351
num: 361
num: 371
num: 381
num: 391
num: 401
num: 411
num: 421
num: 431
num: 441
num: 451
num: 461
num: 471
num: 481
num: 491
num: 501
num: 511
num: 521
num: 531
num: 541
num: 551
num: 561
num: 571
num: 581
num: 591
num: 601
num: 611
num: 621
num: 631
num: 641
num: 651
num: 661
num: 671
num: 681
num: 691
num: 701
num: 711
num: 721
num: 731
num: 741
num: 751
num: 761
num: 771
num: 781
num: 791
num: 801
num: 811
num: 821
num: 831
num: 841
num: 851
num: 861
num: 871
len(lis): 0
len(link_list): 860


In [32]:
driver.quit()

In [33]:
f = open(DATA_DIR+'./naver_link_list.txt','w', encoding='utf-8', newline='')
wr = csv.writer(f)
for i, s in enumerate(link_list):
    wr.writerow([i, s])
f.close()

In [34]:
link_list[:3]

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=023&aid=0003497448',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=023&aid=0003497666',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=023&aid=0003497806']

In [36]:
news_data = []
for i in tqdm_notebook(link_list):
    news_data.append(news_parsing(i, media_list))

<ipython-input-36-4aad1b3ead34>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(link_list):


In [37]:
news_data

[['조선일보',
  '2020-01-02',
  '150만명 몰린  타임스스퀘어 새해 무대도 BTS',
  '  2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계    전 지구를 홀린 그룹입니다   사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 미국 뉴욕 맨해튼 타임스스퀘어 무대에 선 것이다. 12월 31일현지 시각 저녁 열리는 타임스스퀘어 볼드롭대형 크리스털 볼이 신년 카운트다운과 함께 떨어지는 행사을 보기 위해 몰려든 사람들로 이곳은 오전부터 부산스러웠다. 매년 이날만 되면 최소 100만명이 찾는 곳이지만 올해는 BTS 때문에 열기가 더욱 뜨거웠다. 뉴욕경찰NYPD은 이날 몰린 인파가 150만명이라고 추정했다.   한국 가수로 타임스스퀘어 새해맞이 무대에 오른 건 2012년 싸이에 이어 두 번째다. BTS는 밤 10시 38분부터 8분간 열정적인 춤과 노래로 시선을 사로잡았다. 8년 전 100만 인파가 싸이의 강남스타일에 맞춰 말춤을 췄던 것처럼 이날 타임스스퀘어에 모인 전 세계 아미BTS 팬들은 BTS가 작은 것들을 위한 시와 Make It Right를 부를 때 야광봉을 흔들며 한국어로 떼창을 선사했다. 공연을 마친 뒤 리더 RM은 여섯 살 때부터 나 홀로 집에 같은 영화에서 보던 광경이 눈앞에 있다며 감격했다. 멤버들은 한목소리로 해피 뉴 이어를 외쳤다.   BTS 공연은 미국 최대 새해맞이 라이브 쇼인 ABC방송의 뉴 이어스 로킹 이브New Years Rocking Eve를 통해서도 생중계됐다. 최정상급 가수들만 출연하는 유명 프로그램이다. 올해는 BTS 외에 컨트리 가수 샘 헌트와 싱어송 라이터 앨러니스 모리세트가 무대에 섰다. BTS는 2017년 사전 녹화를 통해 할리우드 무대에 출연했지만 타임스스퀘어 무대에 직접 선 것은 이번이 처음이다. 행사를 공동 진행한 라이언 시크레스트는 올해 타임스스퀘어의 절반은 BTS 팬으로 채워질 것이라고 언급했다.   BTS의 인기를 입증하듯 이날 오후부터 타임스스퀘어

In [38]:
col_name = ['media','date','title','article_original','url']
news_df = pd.DataFrame(news_data,columns = col_name)
news_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계 전 지구를...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정 경자년 새해에도 K팝 스타들...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
855,한겨레,2020-12-26,방탄학BTSology BTS 연구는 이미 시작되었다,방탄소년단BTS의 전세계적인 인기는 이제 낯설지 않다. 2017년 처음 미...,https://news.naver.com/main/read.nhn?mode=LSD&...
856,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
857,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년 잊을 수 없는 문화계의 10일 2020년은 한국 문화계의 저력...,https://news.naver.com/main/read.nhn?mode=LSD&...
858,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 이미 최선을 다 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [39]:
news_df.article_original[0]

'  2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계    전 지구를 홀린 그룹입니다   사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 미국 뉴욕 맨해튼 타임스스퀘어 무대에 선 것이다. 12월 31일현지 시각 저녁 열리는 타임스스퀘어 볼드롭대형 크리스털 볼이 신년 카운트다운과 함께 떨어지는 행사을 보기 위해 몰려든 사람들로 이곳은 오전부터 부산스러웠다. 매년 이날만 되면 최소 100만명이 찾는 곳이지만 올해는 BTS 때문에 열기가 더욱 뜨거웠다. 뉴욕경찰NYPD은 이날 몰린 인파가 150만명이라고 추정했다.   한국 가수로 타임스스퀘어 새해맞이 무대에 오른 건 2012년 싸이에 이어 두 번째다. BTS는 밤 10시 38분부터 8분간 열정적인 춤과 노래로 시선을 사로잡았다. 8년 전 100만 인파가 싸이의 강남스타일에 맞춰 말춤을 췄던 것처럼 이날 타임스스퀘어에 모인 전 세계 아미BTS 팬들은 BTS가 작은 것들을 위한 시와 Make It Right를 부를 때 야광봉을 흔들며 한국어로 떼창을 선사했다. 공연을 마친 뒤 리더 RM은 여섯 살 때부터 나 홀로 집에 같은 영화에서 보던 광경이 눈앞에 있다며 감격했다. 멤버들은 한목소리로 해피 뉴 이어를 외쳤다.   BTS 공연은 미국 최대 새해맞이 라이브 쇼인 ABC방송의 뉴 이어스 로킹 이브New Years Rocking Eve를 통해서도 생중계됐다. 최정상급 가수들만 출연하는 유명 프로그램이다. 올해는 BTS 외에 컨트리 가수 샘 헌트와 싱어송 라이터 앨러니스 모리세트가 무대에 섰다. BTS는 2017년 사전 녹화를 통해 할리우드 무대에 출연했지만 타임스스퀘어 무대에 직접 선 것은 이번이 처음이다. 행사를 공동 진행한 라이언 시크레스트는 올해 타임스스퀘어의 절반은 BTS 팬으로 채워질 것이라고 언급했다.   BTS의 인기를 입증하듯 이날 오후부터 타임스스퀘어 주변 곳곳에선 BTS 노래가 흘러나왔고 행인들이 노래에 맞춰 몸을 흔들었다. 새벽 1시부터 줄 서 기다렸다는

In [40]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             860 non-null    object
 1   date              860 non-null    object
 2   title             860 non-null    object
 3   article_original  860 non-null    object
 4   url               860 non-null    object
dtypes: object(5)
memory usage: 33.7+ KB


In [41]:
news_df.to_excel(DATA_DIR+'/news_df_210222_v05.xlsx', index=False)